<a href="https://colab.research.google.com/github/VadPA/PyTorch/blob/master/Lesson_3_hw_v13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

In [174]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 2 * hidden_dim)
        self.fc2 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.dp = nn.Dropout(0.05)
        self.fc3 = nn.Linear(1 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.bn(x)
        x = self.dp(x)
        x = self.fc4(x)

        return x

In [175]:
class CustomDataset(Dataset):
    def __init__(self, is_train=True):
        super(CustomDataset, self).__init__()
        self.is_train = is_train
        self.sc = MinMaxScaler()
        self.X, self.Y = fetch_california_housing(return_X_y=True, as_frame=True)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y, test_size=0.25,
                                                                                random_state=13)

    def __len__(self):
        if self.is_train:
            return len(self.X_train)
        else:
            return len(self.X_test)

    def __getitem__(self, idx):
        if self.is_train:
            x = np.array(self.X_train).astype(np.float32)
            y = np.array(self.y_train).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]
        else:
            x = np.array(self.X_test).astype(np.float32)
            y = np.array(self.y_test).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]

In [176]:
X_train = CustomDataset(is_train=True)

In [177]:
train_loader = torch.utils.data.DataLoader(dataset=X_train,
                                           batch_size=64,
                                           shuffle=True)

In [178]:
X_test = CustomDataset(is_train=False)

In [179]:
test_loader = torch.utils.data.DataLoader(dataset=X_test,
                                          batch_size=64,
                                          shuffle=True)

In [180]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [181]:
net = Net(input_dim=8, hidden_dim=48, output_dim=1).to(device)  # определить сеть
print(net)  # структура сети печати

Net(
  (fc1): Linear(in_features=8, out_features=96, bias=True)
  (fc2): Linear(in_features=96, out_features=48, bias=True)
  (bn): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.05, inplace=False)
  (fc3): Linear(in_features=48, out_features=48, bias=True)
  (fc4): Linear(in_features=48, out_features=1, bias=True)
)


In [182]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001, alpha=0.99)

In [183]:
num_epochs = 5

In [184]:
for epoch in range(num_epochs):
    net.train()
    training_loss = 0.0
    training_items = 0.0
    valid_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # Обнуляем градиент
        optimizer.zero_grad()
        # Делаем предсказание
        outputs = net(inputs)
        # Рассчитываем лосс-функцию
        loss = loss_func(outputs.mH[0, :], labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        training_loss += loss.item()
        training_items += len(labels)
        # печатаем каждые 20 mini-batches
        if i % 20 == 0:
            net.eval()
            print(f'Epoch [{epoch + 1}/{num_epochs}]. '
                  f'Step [{i + 1}/{len(train_loader)}]. '
                  f'Train_Loss: {training_loss / training_items:.4f}', end='. ')
            training_loss, training_items = 0.0, 0.0

            valid_loss, loss_val = 0.0, 0.0
            for j, batch in enumerate(test_loader):
              inputs, targets = batch
              output = net(inputs)
              loss_val = loss_func(output.mH[0, :], targets)
              valid_loss += loss_val.item()
            valid_loss /= len(test_loader)

            print('Valid_Loss: {:.4f}'.format(valid_loss))
            net.train()



Epoch [1/5]. Step [1/242]. Train_Loss: 0.0849. Valid_Loss: 4.8897
Epoch [1/5]. Step [21/242]. Train_Loss: 0.0543. Valid_Loss: 3.5873
Epoch [1/5]. Step [41/242]. Train_Loss: 0.0341. Valid_Loss: 2.1098
Epoch [1/5]. Step [61/242]. Train_Loss: 0.0200. Valid_Loss: 1.1063
Epoch [1/5]. Step [81/242]. Train_Loss: 0.0132. Valid_Loss: 0.9712
Epoch [1/5]. Step [101/242]. Train_Loss: 0.0108. Valid_Loss: 0.5448
Epoch [1/5]. Step [121/242]. Train_Loss: 0.0102. Valid_Loss: 0.5327
Epoch [1/5]. Step [141/242]. Train_Loss: 0.0077. Valid_Loss: 0.4937
Epoch [1/5]. Step [161/242]. Train_Loss: 0.0092. Valid_Loss: 0.5997
Epoch [1/5]. Step [181/242]. Train_Loss: 0.0083. Valid_Loss: 0.7029
Epoch [1/5]. Step [201/242]. Train_Loss: 0.0078. Valid_Loss: 0.5995
Epoch [1/5]. Step [221/242]. Train_Loss: 0.0093. Valid_Loss: 0.7857
Epoch [1/5]. Step [241/242]. Train_Loss: 0.0079. Valid_Loss: 0.6548
Epoch [2/5]. Step [1/242]. Train_Loss: 0.0083. Valid_Loss: 0.5487
Epoch [2/5]. Step [21/242]. Train_Loss: 0.0079. Valid_Lo

In [185]:
data_iter = iter(test_loader)
inputs, labels = data_iter.next()

In [186]:
net.eval()
outputs = net(inputs)

In [187]:
print(outputs.T)

tensor([[1.1937, 1.6008, 4.2049, 4.6409, 1.8449, 3.6507, 1.7184, 1.9777, 2.8583,
         2.8309, 1.0029, 2.3411, 2.1630, 1.4229, 5.0265, 2.9960, 3.1908, 2.0156,
         2.0600, 1.5979, 1.4995, 2.7242, 1.6688, 2.4199, 1.6094, 2.8353, 1.6869,
         1.5160, 3.0222, 2.1628, 1.9510, 2.0621, 2.5807, 3.0331, 2.6155, 1.3639,
         1.7801, 2.7060, 1.5225, 1.4767, 2.0425, 2.6348, 1.8040, 1.5053, 3.3670,
         2.0024, 3.7281, 4.0205, 1.0720, 1.7748, 1.6159, 1.7825, 2.3960, 1.3627,
         3.2377, 2.9780, 1.6279, 2.1981, 1.0164, 1.4905, 3.6971, 1.7358, 1.4234,
         2.6331]], grad_fn=<PermuteBackward0>)


In [188]:
print(loss_func(outputs.mH[0, :], labels))

tensor(0.5616, grad_fn=<MseLossBackward0>)


In [189]:
print(labels)

tensor([1.1030, 1.0780, 4.7280, 5.0000, 2.0210, 4.1370, 1.0000, 2.2500, 2.6730,
        2.2580, 0.6460, 1.8620, 3.6880, 1.4870, 5.0000, 3.3980, 2.8320, 3.2240,
        1.8410, 1.1000, 2.5000, 3.3520, 1.6440, 2.6760, 1.7070, 1.6980, 1.3790,
        1.0700, 3.4250, 2.1590, 2.2410, 2.9840, 2.5400, 5.0000, 4.5000, 1.3960,
        1.3100, 5.0000, 1.2830, 1.2290, 1.7500, 3.2280, 1.0890, 0.9730, 3.4510,
        1.9740, 1.3750, 3.7200, 0.5320, 1.8140, 1.4570, 0.8890, 1.1880, 0.9730,
        2.8750, 2.4350, 2.0170, 2.4030, 0.7800, 1.5770, 5.0000, 1.7050, 1.1690,
        2.9210])


In [190]:
# !pip install torchmetrics

In [191]:
from torchmetrics import R2Score

In [192]:
r2score = R2Score()

In [193]:
r2score(outputs[:,0], labels)

tensor(0.6352, grad_fn=<SqueezeBackward0>)

## Рассчитаем MSE для всего тестового набора

In [194]:
net.eval()
test_loss, loss_test = 0.0, 0.0
for j, batch in enumerate(test_loader):
  inputs, targets = batch
  output = net(inputs)
  loss_test = loss_func(output.mH[0, :], targets)
  test_loss += loss_test.item()
  
  
  
test_loss /= len(test_loader)
print('Test_Loss: {:.4f}'.format(test_loss))

Test_Loss: 0.5237


## Рассчитаем R2 SCORE для всего тестового набора   

In [195]:
output = net(torch.from_numpy(X_test[:][0]).type(torch.FloatTensor))

In [196]:
r2score(output[:,0], torch.from_numpy(X_test[:][1]).type(torch.FloatTensor))

tensor(0.6120, grad_fn=<SqueezeBackward0>)